# Superposition

In this notebook, I'll look at truly testing the superposition aspect of the newer V1 experiments. In particular, for a given stimulus with dispersion > 1 (i.e. not just a single grating), the components of that stimulus will have been presented in isolation. This allows us to test $R_{1+2+..}$ against $R_1 + R_2 + ...$, where $_i$ are different stimulus components.

In [10]:
import os
import numpy as np
import matplotlib
# matplotlib.use('TkAgg') # to avoid GUI/cluster issues...
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as pltSave
import matplotlib.animation as anim
import matplotlib.cm as cm
import seaborn as sns
import itertools
import helper_fcns as hf
import autoreload
import scipy.optimize as opt
from scipy.stats.mstats import gmean as geomean

import sys # so that we can import model_responses (in different folder)
import model_responses

import warnings
warnings.filterwarnings('once');

%matplotlib inline

plt.style.use('https://raw.githubusercontent.com/paul-levy/SF_diversity/master/paul_plt_style.mplstyle');

basePath = os.getcwd() + '/'

## edit
expDir   = 'V1/';
dataListNm = 'dataList_glx.npy'

## now, let it run
dataPath = basePath + expDir + 'structures/'
save_loc = basePath + expDir + 'figures/'

dataList = hf.np_smart_load(dataPath + dataListNm);

### Example cell

In [ ]:
which_cell = 7; # which cell - index will be which_cell - 1
cellName = dataList['unitName'][which_cell-1];
expInd = hf.get_exp_ind(dataPath, cellName)[0]
S = hf.np_smart_load(dataPath + cellName + '_sfm.npy')
expData = S['sfm']['exp']['trial'];

In [ ]:
### organize responses
resps, stimVals, val_con_by_disp, _, _ = hf.tabulate_responses(expData, expInd);
### WARNING: WARNING: resps is WITHOUT any rvcAdjustment
predResps = resps[2];
rvcFits = hf.get_rvc_fits(dataPath, expInd, which_cell, rvcName='None');
spikes  = hf.get_spikes(expData, rvcFits=rvcFits, expInd=expInd);
_, _, respOrg, respAll    = hf.organize_resp(spikes, expData, expInd);

respMean = respOrg;
respStd = np.nanstd(respAll, -1); # take std of all responses for a given condition
# compute SEM, too
findNaN = np.isnan(respAll);
nonNaN  = np.sum(findNaN == False, axis=-1);
respSem = np.nanstd(respAll, -1) / np.sqrt(nonNaN);

### organize stimulus information
all_disps = stimVals[0];
all_cons = stimVals[1];
all_sfs = stimVals[2];

nCons = len(all_cons);
nSfs = len(all_sfs);
nDisps = len(all_disps);

In [ ]:
maxResp = np.maximum(np.nanmax(respMean), np.nanmax(predResps));
# by disp
clrs_d = cm.viridis(np.linspace(0,1,nDisps-1));
lbls_d = ['disp: %s' % str(x) for x in range(nDisps)];
# by sf
val_sfs = hf.get_valid_sfs(S, d=1, val_con_by_disp[1][0]) # pick 
clrs_d = cm.viridis(np.linspace(0,1,nDisps-1));
lbls_d = ['disp: %s' % str(x) for x in range(nDisps)];

f, ax = plt.subplots(1, 2, figsize=(25, 10))

allMix = [];
allSum = [];

for d in range(nDisps):
    if d == 0: # we don't care about single gratings!
        continue; 
    v_cons = np.array(val_con_by_disp[d]);
    n_v_cons = len(v_cons);

    for c in reversed(range(n_v_cons)):
        v_sfs = hf.get_valid_sfs(S, d, v_cons[c], expInd)
        for s in v_sfs:
            mixResp = respMean[d, s, v_cons[c]];
            allMix.append(mixResp);
            sumResp = predResps[d, s, v_cons[c]];
            allSum.append(sumResp);
#             print('condition: d(%d), c(%d), sf(%d):: pred(%.2f)|real(%.2f)' % (d, v_cons[c], s, sumResp, mixResp))
            if c == 0 and s == v_sfs[0]:
                ax[0].plot(sumResp, mixResp, 'o', color=clrs_d[d-1], label=lbls_d[d])
            else:
                ax[0].plot(sumResp, mixResp, 'o', color=clrs_d[d-1])
   
# make a polynomial fit
hmm = np.polyfit(allSum, allMix, deg=1) # returns [a, b] in ax + b 

for i in range(2):
    ax[i].axis('equal')
    ax[i].set_xlabel('predicted');
    ax[i].set_ylabel('superposition');
    ax[i].plot([0, 1.2*maxResp], [0, 1.2*maxResp], 'k--')
    ax[i].set_title('Suppression index: %.2f' % hmm[0])
    ax[i].legend();

### Full analysis

Now, let's gather the same pred/mix responses for all cells

In [11]:
suppInds = [];
for nm, ind in zip(dataList['unitName'], range(len(dataList['unitName']))):
    
    try:
        # load cell
        expInd = hf.get_exp_ind(dataPath, nm)[0]
        S = hf.np_smart_load(dataPath + nm + '_sfm.npy')
        expData = S['sfm']['exp']['trial'];
    except:
        continue;
    
    ### organize responses
    resps, stimVals, val_con_by_disp, _, _ = hf.tabulate_responses(expData, expInd);
    ### WARNING: WARNING: resps is WITHOUT any rvcAdjustment
    predResps = resps[2];
    rvcFits = hf.get_rvc_fits(dataPath, expInd, ind, rvcName='None');
    spikes  = hf.get_spikes(expData, rvcFits=rvcFits, expInd=expInd);
    _, _, respOrg, respAll    = hf.organize_resp(spikes, expData, expInd);

    respMean = respOrg;
    respStd = np.nanstd(respAll, -1); # take std of all responses for a given condition                                                                                                                         
    # compute SEM, too                                                                                                                                                                                          
    findNaN = np.isnan(respAll);
    nonNaN  = np.sum(findNaN == False, axis=-1);
    respSem = np.nanstd(respAll, -1) / np.sqrt(nonNaN);

    ### organize stimulus information
    all_disps, all_cons, all_sfs = stimVals[0:3];
    nCons, nSfs, nDisps = len(all_cons), len(all_sfs), len(all_disps);
    
    allMix = [];
    allSum = [];

    for d in range(nDisps):
        if d == 0: # we don't care about single gratings!
            continue; 
        v_cons = np.array(val_con_by_disp[d]);
        n_v_cons = len(v_cons);

        for c in reversed(range(n_v_cons)):
            v_sfs = hf.get_valid_sfs(S, d, v_cons[c], expInd)
            for s in v_sfs:
                mixResp = respMean[d, s, v_cons[c]];
                allMix.append(mixResp);
                sumResp = predResps[d, s, v_cons[c]];
                allSum.append(sumResp);
    
    hmm = np.polyfit(allSum, allMix, deg=1) # returns [a, b] in ax + b 
    suppInds.append(hmm[0])
    

/arc/2.2/p1/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/helper_fcns.py:1493: UserWarning: This experiment type (expInd=3) usually has associated RVC fits for resposne adjustment
  warnings.warn('This experiment type (expInd=3) usually has associated RVC fits for resposne adjustment');
/arc/2.2/p1/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/helper_fcns.py:1462: RuntimeWarning: Mean of empty slice
  rateSfMix = numpy.nanmean(allSfMix, -1);
/users/plevy/.conda/envs/lcv-python/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
